# Homeostasis

# Synaptic Normalisation

Scaling the synapses allows the postsynaptic neuron to maintain a favourable firing rate while still allowing its inputs to change due to experience.
Besides STDP, there exist forms of synaptic plasticity that aim at restoring or maintaining a certain activity level in a homeostatic manner. For instance, through homeostatic plasticity, an overly active neuron may weaken its excitatory inputs or an overly silent neuron may strengthen its excitatory inputs. The effect of homeostasis on the activity of a neuron or network of neurons depends strongly on its timescale. Some forms of homeostatic plasticity rescale synapses over hour- or day-long timescales in a multiplicative manner, as shown  in rodents after growth or shrinkage of synaptic sizes during experience-dependent synaptic plasticity {cite}`Turrigiano1998` {cite}`Keck2013`. We refer to this slow homeostatic plasticity as ''synaptic scaling''.
On the other hand, there is also fast synaptic homeostasis {cite}`Bourne2011`, which is able to keep runaway excitation under control {cite}`Zenke2013`. The fast synaptic homeostatic plasticity is most likely heterosynaptic, meaning that the change in one synaptic weight depends on the collective dynamics or states of synapses that are connected to the same postsynaptic target. We call the fast homeostatic plasticity ''synaptic normalisation''. Until more data becomes available, we assume that synaptic normalisation acts multiplicatively {cite}`Triesch2018`, like synaptic scaling. The major advantage of multiplicative synaptic normalisation is that the proportional difference between the smaller and larger weights is conserved. Multiplicative synaptic normalisation therefore does not erase pre-existing memories if it is implemented in a network of neurons. 
An example of multiplicative synaptic normalisation is the following:

$$
w_{j,t + 1} = w_{j, t} \bigg (  \frac{W_\mathrm{tot}}{\sum^{N}_{k=1} w_{k,t}} \bigg ) \; ,
$$ (weight_norm_eq)

Where $j$ and $k$ refer to each synapse onto the neuron, and $N$ is the total number of synapses onto the neuron. The parameter $W_\mathrm{tot}$ is the value of the total allowed weight onto the neuron, which can be assumed to be flexible over time but we here keep fixed for simplicity. Let us say we wish to have a normalised weight at timestep $t+1$ in the simulation. The weight $w_{j, t}$ then is the weight just before a normalisation event. After applying equation {eq}`weight_norm_eq`, we obtain the weight just after normalisation, $w_{j,t + 1}$, for the timestep $t + 1$. We assume in this case that normalisation events occur periodically, for instance once every second in simulated time.

One can easily see from the equation above that the weight is scaled up or down, depending on whether the sum of the weights is below or above the target value $W_\mathrm{tot}$. Each individual weight $w_j$ of course contributes to that sum. However, there is one problem with this equation: it permits instantaneous change of any size to a single weight, which may be very large if the weight is far removed from its target value after scaling. One could argue that such a large instantaneous change is not very realistic in a biological neuron, where a given synapse cannot undergo arbitrarily large changes in a small timeframe. We therefore make a small modification, adding a parameter $\eta_{\mathrm{SN}}$ that represents the proportion of change from the weight prior to scaling. In other words, for $\eta_{\mathrm{SN}}$ the weight will still move towards its value dictated by complete scaling, but will actually only change by for example 20 percent at every scaling event. By modifying the equation above, we obtain

$$
w_{j,t + 1} = w_{j, t}\bigg(1 +  \eta_{\mathrm{SN}}  \bigg (  \frac{W_\mathrm{tot}}{\sum^{N}_{k=1} w_{k,t}} -1 \bigg ) \bigg ) \; .  
$$ (weight_norm_modified_eq)

The synaptic normalisation rate $\eta_{\mathrm{SN}}$ can be set from any value between 0 and 1, where 1 results in reinstating $W_\mathrm{tot}$ precisely at each normalisation, equation 19. 
For our model neuron, let us use $\eta_{\mathrm{SN}}$ = 0.2. For practicality, you can implement the normalisation in the neuron simulation to happen every few seconds, depending on how long you simulate your neuron. Synaptic normalisation is also synapse-type specific, occurring separately for all excitatory and all inhibitory weights.
Provide the neuron with excitatory inputs from two groups of each 50 spike trains with spike correlations $c$=0.1 for group 1 and $c$ = 0.2 for group 2, as in the previous section, with STDP, and endow all excitatory synapses with normalisation. Start the weights at 0.1, and use $W_\mathrm{tot}$ = 3. Also include 30 inhibitory Poisson inputs, with $w_\mathrm{i} = $ 1.0 and firing rate 10 Hz, but without STDP and normalisation, for simplicity. 
Normalise the excitatory synapses every second, using equation {eq}`weight_norm_modified_eq`. Run a simulation for 30 seconds. What is happening to the firing rate of the neuron? Is there still competition between the weights of the two input groups? How do the weight distributions of the two groups look at the end of the simulation?

In [ ]:
4_1_STDP_norm_wevol.png
4_1_STDP_norm_wdist.png
A LIF neuron with two excitatory input groups with STDP and synaptic normalisation in its synapses. The normalisation is applied once every second here. Top left: The LIF neuron is more depolarised over time. Top right: Evolution of the synaptic weights from both input groups. The thin lines show the individual weights, the thick lines the mean weight of the corresponding group. Bottom: The histograms of the weights from each group at the end of the simulation are shown.

## Intrinsic plasticity

Experiments have shown that neurons have multiple strategies for maintaining healthy firing rates. Some types of plasticity can regulate the neuron's output spiking without acting on the input synapses. We call these "Intrinsic Plasticity" (IP) mechanisms {cite}`Desai1999`. Instead of changing the structure and protein content of the synapses, IP acts upon the ion channels in the neuron membrane, making the neuron less responsive to a single input if the strength of the inputs increases. 
The IP can therefore implement a homeostatic effect, nudging the spiking threshold of the neuron down or up depending on its recent history of activity.
The time scale of IP is typically on the order of hours or even days. In a model neuron, we do not necessarily employ such a long timescale directly, but as with synaptic normalisation, it should be a slower dynamic compared to STDP. 
In the LIF neuron, IP can be directly implemented with a moving threshold $V_\mathrm{thresh}$, and a target firing rate for the neuron, $R_\mathrm{target}$. 
First, create an online spike counter that records the neuron's output spikes, and determine the online firing rate in Hz every second as $R_\mathrm{count}$. Set the neuron to change its $V_\mathrm{thresh}$ by an amount $\eta_\mathrm{IP}$ = 0.1 mV every second. Reset $R_\mathrm{count}$ right after every change to $V_\mathrm{thresh}$. When the spike counter produces a value that deviates from the target firing rate,

$$
V_{\mathrm{thresh},t + 1} = V_{\mathrm{thresh},t} + \eta_\mathrm{IP} (R_{\mathrm{count},t} - R_\mathrm{target}).
$$ (sliding_thresh_eq)

For instance, $V_\mathrm{thresh}$ is shifted upwards when the firing rate is too high, and as a consequence the neuron responds less to the same inputs. This is then an opposite effect to for instance, growing synaptic weights from excitatory inputs.
Leave out STDP and normalisation in this exercise. Test the IP in a neuron receiving 10 excitatory spike trains with frequency 3 Hz, and a target postsynaptic firing rate of 3 Hz. Set the initial $w_\mathrm{e}$ to 0.35. Add STDP with $A_\mathrm{LTP} =$ 0.001 and $A_\mathrm{LTD} =$ = -0.0005. Add also 10 inhibitory Poisson inputs of 10 Hz with $w_\mathrm{i} = $ 1.0. To see the effect best, remove the synaptic normalisation included before. Are the excitatory weights increasing? Does the neuron maintain its firing rate over time? Does the change in firing rate affect the weight evolution?

4_2_IP.png
A LIF neuron with an adapable spiking threshold (intrinsic plasticity) and Poisson spiking inputs with STDP. Left: The synaptic weights from the two excitatory input groups increase over time due to STDP. Middle: The spiking threshold first decreases to meet the target firing rate, then increases to compensate for the strong synaptic inputs at the end of the simulation. Right: The output firing rate becomes stable over time.}

